1、实现K近邻算法

2、采用K近邻对1个数据集进行分类分析，进行分类训练和测试，并给出分析结果

注意：作业提交需包含所有代码，必要时可提交压缩包，代码尽可能在(姓名_KNN.ipynb)中

数据集可以采用阿尔及利亚森林火灾数据集：UCI Machine Learning Repository: Algerian Forest Fires Dataset Data Set

In [50]:
import pandas as pd 
import numpy as np

In [58]:
df=pd.read_csv('./dataset/Algerian Forest Fires Dataset Data Set/Algerian_forest_fires_dataset_UPDATE.csv',header=1)
df.head()
df=df.drop([122,123,124])
df.iloc[121:125]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
121,30,9,2012,25,78,14,1.4,45,1.9,7.5,0.2,2.4,0.1,not fire
125,1,6,2012,32,71,12,0.7,57.1,2.5,8.2,0.6,2.8,0.2,not fire
126,2,6,2012,30,73,13,4,55.7,2.7,7.8,0.6,2.9,0.2,not fire
127,3,6,2012,29,80,14,2,48.7,2.2,7.6,0.3,2.6,0.1,not fire


In [59]:
df=df.reset_index(drop=True)
df.iloc[121:125]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
121,30,9,2012,25,78,14,1.4,45,1.9,7.5,0.2,2.4,0.1,not fire
122,1,6,2012,32,71,12,0.7,57.1,2.5,8.2,0.6,2.8,0.2,not fire
123,2,6,2012,30,73,13,4,55.7,2.7,7.8,0.6,2.9,0.2,not fire
124,3,6,2012,29,80,14,2,48.7,2.2,7.6,0.3,2.6,0.1,not fire


In [60]:
# 对数据进行重新编码
df['Classes  ']=df['Classes  '].apply(lambda x:1 if x=='not fire   ' else 0)
df['Classes  '].value_counts()

0    143
1    101
Name: Classes  , dtype: int64

In [61]:
# 去除年份列，其只有一年
df=df.drop(columns='year')
df

,day,month,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
0,1,6,29,57,18,0,65.7,3.4,7.6,1.3,3.4,0.5,1
1,2,6,29,61,13,1.3,64.4,4.1,7.6,1,3.9,0.4,1
2,3,6,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,1
3,4,6,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,1
4,5,6,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,26,9,30,65,14,0,85.4,16,44.5,4.5,16.9,6.5,0
240,27,9,28,87,15,4.4,41.1,6.5,8,0.1,6.2,0,1
241,28,9,27,87,29,0.5,45.9,3.5,7.9,0.4,3.4,0.2,1
242,29,9,24,54,18,0.1,79.7,4.3,15.2,1.7,5.1,0.7,1


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          244 non-null    object
 1   month        244 non-null    object
 2   Temperature  244 non-null    object
 3    RH          244 non-null    object
 4    Ws          244 non-null    object
 5   Rain         244 non-null    object
 6   FFMC         244 non-null    object
 7   DMC          244 non-null    object
 8   DC           244 non-null    object
 9   ISI          244 non-null    object
 10  BUI          244 non-null    object
 11  FWI          244 non-null    object
 12  Classes      244 non-null    int64 
dtypes: int64(1), object(12)
memory usage: 24.9+ KB


In [71]:
X=df.iloc[:,:-1]
X=X.values
X=X.astype(float)

In [72]:
import numpy as np

class KNN(object):
    def __init__(self, k):
        self.k = k   # 初始化k值，即最近邻居的数量

    def fit(self, X_train, y_train):
        self.X_train = X_train  # 训练数据集
        self.y_train = y_train  # 训练标签

    def predict(self, X_test):
        predictions = []
        for i in range(len(X_test)):
            distances = np.sqrt(np.sum(np.square(self.X_train - X_test[i, :]), axis=1))  # 计算测试样本和所有训练样本之间的距离
            sorted_indices = np.argsort(distances)  # 对距离进行排序
            k_nearest_labels = [self.y_train[j] for j in sorted_indices[:self.k]]  # 选取最近的k个样本的标签
            counts = np.bincount(k_nearest_labels)  # 统计每种标签的出现次数
            predictions.append(np.argmax(counts))  # 选择出现次数最多的标签作为预测结果
        return predictions


In [73]:
 # 划分训练集和测试集，设置种子可重复实验
from sklearn.model_selection import train_test_split
y=df.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [76]:
model = KNN(k=3)
model.fit(X_train, y_train)
y_prob = model.predict(X_test)